Following the work done by 
http://www.marekrei.com/blog/theano-tutorial/ 

In [16]:
from IPython.core.debugger import set_trace
import numpy as np
import theano as T

Example 1)

Creating the squeleton (theano objects). Here nothing is calculated. Creating the "graph". 
Usually the **variables** defined as with X, are variables that do not have any values. Tipically they will be used as for inputs to our network.<br>
**Shared variables** which are shared between different functions (weights in NN). WIll be moved to GPU automatically.



In [49]:
x=T.tensor.vector('xa',dtype='float32')
#x=T.tensor.fvector('x')
W=T.shared(np.asarray([0.2,0.7]),'W')
y=(x*W).sum()

W.get_value()


array([0.2, 0.7])

Performing the real operation thanks to the **functions**  that create a link between the graph and the user. Define first a list of inputs and geta list of outputs.


In [51]:
f = T.function([x], y)
output = f([1.0, 1.0])
print("Value of the output with first set of W",output)

W.set_value([0.3,0.9])
print("Changing the value of W to...",W.get_value())
output = f([1.0, 1.0])
print("Value of the output with second set of W--->",output)



Value of the output with first set of W 0.8999999999999999
Changing the value of W to... [0.3 0.9]
Value of the output with second set of W---> 1.2


# Training example <br>
A simple code where a toy example of training (change the weights) is provided.<br>
### Gradients
In order to calculate the partial derivatives with respect to the lost function, a function provided by Theano is used. A **scalar** and a list of **variables** is provided to the function (the list of variables we want gradients for) ---> Return a _list_ <br>

### List of Updates
First a symbolic variable to store the **deltaW** is created based on the _gradients_ and the _learning rate_. Then, we create a **list of tuples** (Theano requires this) where the **_first element_** is the variable we want to update, and the second one is the values that the variable will have after the update. <br>

### Function

It now takes two input arguments – one for the input vector, and another for the target value used for training. And the list of updates also gets attached to the function as well. Every time this function is called, we pass in values for x and target, get back the value for y, the gradients and the actual weights (we do not really need any of these values, is just to check how is  working) as outputs, and Theano performs all the updates in the update list.




In [67]:
#Define the squeleton
x=T.tensor.fvector('x')
target=T.tensor.fscalar('target')
W=T.shared(np.asarray([0.2,0.7]),'W')
y=(x*W).sum()

cost=T.tensor.sqr(target-y)
gradients=T.tensor.grad(cost,[W])
W_updated=W-(0.01*gradients[0])
updates=[(W,W_updated)]

#define the function (a bit more "complex" now)
f=T.function([x,target],[y,gradients[0],W],updates=updates)

for i in range(1000):
    [output,grad,weigh]=f([1.0,1.0],20.0)
    print(output,grad,weigh)


0.8999999999999999 [-38.2 -38.2] [0.2 0.7]
1.6640000000000001 [-36.672 -36.672] [0.582 1.082]
2.39744 [-35.20512 -35.20512] [0.94872 1.44872]
3.1015424 [-33.7969152 -33.7969152] [1.3007712 1.8007712]
3.7774807040000002 [-32.44503859 -32.44503859] [1.63874035 2.13874035]
4.42638147584 [-31.14723705 -31.14723705] [1.96319074 2.46319074]
5.0493262168064 [-29.90134757 -29.90134757] [2.27466311 2.77466311]
5.647353168134144 [-28.70529366 -28.70529366] [2.57367658 3.07367658]
6.221459041408778 [-27.55708192 -27.55708192] [2.86072952 3.36072952]
6.772600679752427 [-26.45479864 -26.45479864] [3.13630034 3.63630034]
7.30169665256233 [-25.39660669 -25.39660669] [3.40084833 3.90084833]
7.809628786459838 [-24.38074243 -24.38074243] [3.65481439 4.15481439]
8.297243635001443 [-23.40551273 -23.40551273] [3.89862182 4.39862182]
8.765353889601386 [-22.46929222 -22.46929222] [4.13267694 4.63267694]
9.21473973401733 [-21.57052053 -21.57052053] [4.35736987 4.85736987]
9.646150144656637 [-20.70769971 -20.7

19.99999883741367 [-2.32517266e-06 -2.32517266e-06] [ 9.74999942 10.24999942]
19.999998883917122 [-2.23216576e-06 -2.23216576e-06] [ 9.74999944 10.24999944]
19.999998928560437 [-2.14287913e-06 -2.14287913e-06] [ 9.74999946 10.24999946]
19.99999897141802 [-2.05716396e-06 -2.05716396e-06] [ 9.74999949 10.24999949]
19.9999990125613 [-1.9748774e-06 -1.9748774e-06] [ 9.74999951 10.24999951]
19.999999052058847 [-1.89588231e-06 -1.89588231e-06] [ 9.74999953 10.24999953]
19.999999089976495 [-1.82004701e-06 -1.82004701e-06] [ 9.74999954 10.24999954]
19.999999126377435 [-1.74724513e-06 -1.74724513e-06] [ 9.74999956 10.24999956]
19.99999916132234 [-1.67735532e-06 -1.67735532e-06] [ 9.74999958 10.24999958]
19.999999194869446 [-1.61026111e-06 -1.61026111e-06] [ 9.7499996 10.2499996]
19.999999227074667 [-1.54585067e-06 -1.54585067e-06] [ 9.74999961 10.24999961]
19.999999257991682 [-1.48401664e-06 -1.48401664e-06] [ 9.74999963 10.24999963]
19.999999287672015 [-1.42465597e-06 -1.42465597e-06] [ 9.7499

19.99999999999992 [-1.63424829e-13 -1.63424829e-13] [ 9.75 10.25]
19.999999999999922 [-1.56319402e-13 -1.56319402e-13] [ 9.75 10.25]
19.999999999999925 [-1.49213975e-13 -1.49213975e-13] [ 9.75 10.25]
19.99999999999993 [-1.42108547e-13 -1.42108547e-13] [ 9.75 10.25]
19.999999999999932 [-1.3500312e-13 -1.3500312e-13] [ 9.75 10.25]
19.999999999999936 [-1.27897692e-13 -1.27897692e-13] [ 9.75 10.25]
19.99999999999994 [-1.20792265e-13 -1.20792265e-13] [ 9.75 10.25]
19.999999999999943 [-1.13686838e-13 -1.13686838e-13] [ 9.75 10.25]
19.999999999999947 [-1.0658141e-13 -1.0658141e-13] [ 9.75 10.25]
19.99999999999995 [-9.9475983e-14 -9.9475983e-14] [ 9.75 10.25]
19.999999999999954 [-9.23705556e-14 -9.23705556e-14] [ 9.75 10.25]
19.999999999999957 [-8.52651283e-14 -8.52651283e-14] [ 9.75 10.25]
19.999999999999957 [-8.52651283e-14 -8.52651283e-14] [ 9.75 10.25]
19.999999999999957 [-8.52651283e-14 -8.52651283e-14] [ 9.75 10.25]
19.999999999999957 [-8.52651283e-14 -8.52651283e-14] [ 9.75 10.25]
19.99